In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = ["loan_amnt", "int_rate", "annual_inc", "open_acc", "total_acc", "total_pymnt", "loan_status"]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,annual_inc,open_acc,total_acc,total_pymnt,loan_status
0,35000.0,0.1308,125890.0,14.0,29.0,35016.528333,Fully Paid
1,20000.0,0.2250,75000.0,8.0,11.0,719.000000,low_risk
2,10500.0,0.1719,66000.0,8.0,14.0,355.290000,low_risk
3,25000.0,0.2000,105000.0,17.0,34.0,873.530000,low_risk
4,20000.0,0.2000,56000.0,8.0,21.0,485.440000,low_risk


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [7]:
X.describe()

,loan_amnt,int_rate,annual_inc,open_acc,total_acc,total_pymnt
count,96840.000000,96840.000000,9.684000e+04,96840.000000,96840.000000,96840.000000
mean,16642.454822,0.126793,8.516747e+04,11.707239,23.079864,1272.141466
std,10365.336053,0.048470,1.133767e+05,5.978739,12.186639,2796.006935
min,1000.000000,0.060000,0.000000e+00,0.000000,2.000000,0.000000
25%,9000.000000,0.081900,4.900000e+04,7.000000,14.000000,408.060000
50%,15000.000000,0.118000,7.000000e+04,11.000000,21.000000,726.530000
75%,24000.000000,0.155700,1.000000e+05,15.000000,30.000000,1258.670000
max,40000.000000,0.308900,9.000000e+06,72.000000,164.000000,41418.981165


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk       94116
Fully Paid      2157
high_risk        526
Charged Off       41
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 70569,
         'Fully Paid': 1644,
         'high_risk': 385,
         'Charged Off': 32})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.25

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    0,     0,     0,     9],
       [    0,     0,     0,   513],
       [    0,     0,     0,   141],
       [    0,     0,     0, 23547]], dtype=int64)

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.00      0.00      1.00      0.00      0.00      0.00         9
 Fully Paid       0.00      0.00      1.00      0.00      0.00      0.00       513
  high_risk       0.00      0.00      1.00      0.00      0.00      0.00       141
   low_risk       0.97      1.00      0.00      0.99      0.00      0.00     23547

avg / total       0.95      0.97      0.03      0.96      0.00      0.00     24210



In [22]:
# List the features sorted in descending order by feature importance
importances = clf.feature_importances_

sorted(zip(clf.feature_importances_, X.columns), reverse=True)

# NEED TO CHANGE HOW THIS IS PRINTED: going to print as an array rn with column name second

[(0.18, 'int_rate'),
 (0.08, 'total_acc'),
 (0.08, 'loan_amnt'),
 (0.04, 'open_acc'),
 (0.04, 'annual_inc'),
 (0.02, 'total_pymnt')]

### Easy Ensemble AdaBoost Classifier

In [16]:
# Train the EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

X_train, y_train = make_classification(random_state=1)
clf = AdaBoostClassifier(random_state=0)
clf.fit(X_train, y_train)
AdaBoostClassifier(random_state=1)
clf.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
clf.score(X_train, y_train)

1.0

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

ValueError: X has 6 features, but AdaBoostClassifier is expecting 20 features as input.

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    0,     0,     0,     9],
       [    0,     0,     0,   513],
       [    0,     0,     0,   141],
       [    0,     0,     0, 23547]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.00      0.00      1.00      0.00      0.00      0.00         9
 Fully Paid       0.00      0.00      1.00      0.00      0.00      0.00       513
  high_risk       0.00      0.00      1.00      0.00      0.00      0.00       141
   low_risk       0.97      1.00      0.00      0.99      0.00      0.00     23547

avg / total       0.95      0.97      0.03      0.96      0.00      0.00     24210

